## a. Download CIFAR-10 dataset

In [ ]:
import os
import re
import sys
import tarfile
from six.moves import urllib

def maybe_download_and_extract(
  dest_directory='data',
  data_url='http://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz'):
  
  """Download and extract the tarball from Alex's website."""
  dest_directory = dest_directory
  if not os.path.exists(dest_directory):
    os.makedirs(dest_directory)
  filename = data_url.split('/')[-1]
  filepath = os.path.join(dest_directory, filename)
  if not os.path.exists(filepath):
    def _progress(count, block_size, total_size):
      sys.stdout.write('\r>> Downloading %s %.1f%%' % (
          filename, float(count * block_size) / float(total_size) * 100.0))
      sys.stdout.flush()
    filepath, _ = urllib.request.urlretrieve(DATA_URL, filepath, _progress)
    print()
    statinfo = os.stat(filepath)
    print('Successfully downloaded', filename, statinfo.st_size, 'bytes.')
  extracted_dir_path = os.path.join(dest_directory, 'cifar-10-batches-bin')
  if not os.path.exists(extracted_dir_path):
    tarfile.open(filepath, 'r:gz').extractall(dest_directory)

In [ ]:
DATA_DIR = 'data'
DATA_URL = 'http://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz'
maybe_download_and_extract(DATA_DIR, DATA_URL)

## b. Exploring CIFAR-10 dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def extract_data(index=0, filepath='data/cifar-10-batches-bin/data_batch_5.bin'):
  bytestream = open(filepath, mode='rb')

  label_bytes_length = 1
  image_bytes_length = (32 ** 2) * 3
  record_bytes_length = label_bytes_length + image_bytes_length

  bytestream.seek(record_bytes_length * index, 0)
  label_bytes = bytestream.read(label_bytes_length)
  image_bytes = bytestream.read(image_bytes_length)

  label = np.frombuffer(label_bytes, dtype=np.uint8)  
  image = np.frombuffer(image_bytes, dtype=np.uint8)
  
  image = np.reshape(image, [3, 32, 32])
  image = np.transpose(image, [1, 2, 0])
  image = image.astype(np.float32)
  
  result = {
    'image': image,
    'label': label,
  }
  bytestream.close()
  return result

In [ ]:
%matplotlib inline
result = extract_data(np.random.randint(1000))
plt.imshow(result['image'])

## Steps to use the TF Estimator APIs
1. Define dataset **metadata** and **global constants**
2. Define **data input function** to read the data from the source + **apply pre-processing**
3. Create TF **feature columns** based on metadata + **extended feature columns**
4. Instantiate a **model function** with the required **feature columns, EstimatorSpecs, & parameters**
5. Define a **serving function**
6. Run **Experiment** by supplying training and validation data, as well as required parameters
7. **Evaluate** the model using test data
8. Perform **predictions**

In [ ]:
import math
import shutil
import tensorflow as tf

from datetime import datetime
from tensorflow.python.feature_column import feature_column

from tensorflow.contrib.learn import learn_runner
from tensorflow.contrib.learn import make_export_strategy

print(tf.__version__)

In [ ]:
train_data_files = ['data/cifar-10-batches-bin/data_batch_{}.bin'.format(i) for i in range(1,5)]
valid_data_files = ['data/cifar-10-batches-bin/data_batch_5.bin']
test_data_files = ['data/cifar-10-batches-bin/test_batch.bin']

## 1. Define dataset metadata and global constants

In [ ]:
# Process images of this size. Note that this differs from the original CIFAR
# image size of 32 x 32. If one alters this number, then the entire model
# architecture will change and any model would need to be retrained.
IMAGE_HEIGHT = 32
IMAGE_WIDTH = 32
IMAGE_DEPTH = 3

# Global constants describing the CIFAR-10 data set.
NUM_CLASSES = 10
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 50000
NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = 10000

# If a model is trained with multiple GPUs, prefix all Op names with tower_name
# to differentiate the operations. Note that this prefix is removed from the
# names of the summaries when visualizing a model.
TOWER_NAME = 'tower'

# We use a weight decay of 0.0002, which performs better than the 0.0001 that
# was originally suggested.
WEIGHT_DECAY = 2e-4
MOMENTUM = 0.9

# Global constants describing model behaviors
MODEL_NAME = 'cnn-model-02'
USE_CHECKPOINT = False

## 2. Define Data Input Function

### a. parsing CIFAR-10 dataset

In [ ]:
def parse_record(raw_record):
  # Every record consists of a label followed by the image, with a fixed number
  # of bytes for each.
  label_bytes = 1
  image_bytes = IMAGE_HEIGHT * IMAGE_WIDTH * IMAGE_DEPTH
  record_bytes = label_bytes + image_bytes
  
  # Convert from a string to a vector of uint8 that is record_bytes long.
  record_vector = tf.decode_raw(raw_record, tf.uint8)
  
  # The first byte represents the label, which we convert from uint8 to int32
  # and then to one-hot.
  label = tf.cast(record_vector[0], tf.int32)
  label = tf.one_hot(label, NUM_CLASSES)
  
  # The remaining bytes after the label represent the image, which we reshape
  # from [depth * height * width] to [depth, height, width].
  depth_major = tf.reshape(
    record_vector[label_bytes:record_bytes], [IMAGE_DEPTH, IMAGE_HEIGHT, IMAGE_WIDTH])
  
  # Convert from [depth, height, width] to [height, width, depth], and cast as
  # float32.
  image = tf.cast(tf.transpose(depth_major, [1, 2, 0]), tf.float32)
  
  return image, label

### b. preprocessing CIFAR-10 dataset

In [ ]:
def preprocess_image(image, is_training=False):
  """Preprocess a single image of layout [height, width, depth]."""
  if is_training:
    # Resize the image to add four extra pixels on each side.
    image = tf.image.resize_image_with_crop_or_pad(
        image, IMAGE_HEIGHT + 8, IMAGE_WIDTH + 8)

    # Randomly crop a [_HEIGHT, _WIDTH] section of the image.
    image = tf.random_crop(image, [IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH])

    # Randomly flip the image horizontally.
    image = tf.image.random_flip_left_right(image)

  # Subtract off the mean and divide by the variance of the pixels.
  image = tf.image.per_image_standardization(image)
  return image

### c. data pipeline input function

In [ ]:
def generate_input_fn(file_names,
                      mode=tf.estimator.ModeKeys.EVAL,
                      num_epochs=None,
                      batch_size=1):

  def _input_fn():
    label_bytes = 1
    image_bytes = IMAGE_HEIGHT * IMAGE_WIDTH * IMAGE_DEPTH
    record_bytes = label_bytes + image_bytes
    dataset = tf.data.FixedLengthRecordDataset(filenames=file_names,
                                               record_bytes=record_bytes)

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)
    if is_training:
      buffer_size = batch_size * 2 + 1
      dataset = dataset.shuffle(buffer_size=buffer_size)

    dataset = dataset.map(parse_record)
    dataset = dataset.map(lambda image, label: (preprocess_image(image, is_training), label))

    dataset = dataset.prefetch(2 * batch_size)

    # We call repeat after shuffling, rather than before, to prevent separate
    # epochs from blending together.
    dataset = dataset.repeat(num_epochs)

    # Batch results by up to batch_size, and then fetch the tuple from the
    # iterator.
    dataset = dataset.batch(batch_size)
    iterator = dataset.make_one_shot_iterator()
    images, labels = iterator.get_next()

    features = {'images': images}
    return features, labels
  
  return _input_fn

## 3. Define Feature Columns

In [ ]:
def get_feature_columns():
  feature_columns = {
    'images': tf.feature_column.numeric_column('images', (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH)),
  }
  return feature_columns

In [ ]:
feature_columns = get_feature_columns()
print("Feature Columns: {}".format(feature_columns))

## 4. Instantiate an Estimator

In [ ]:
def get_model():
  model = tf.keras.models.Sequential()
  # Define input tensor in Keras world.
  model.add(tf.keras.layers.InputLayer(input_shape=(IMAGE_WIDTH*IMAGE_HEIGHT*IMAGE_DEPTH,), name='images'))
  model.add(tf.keras.layers.Reshape(target_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_DEPTH)))
  # The first convolutional layer.
  model.add(tf.keras.layers.Conv2D(
      filters=32, kernel_size=(3, 3), padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'))

  # The second convolutional layer.
  model.add(tf.keras.layers.Conv2D(
      filters=32, kernel_size=(3, 3), padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'))
  model.add(tf.keras.layers.Dropout(0.25))
    
  # The third convolutional layer
  model.add(tf.keras.layers.Conv2D(
      filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
    
  # The fourth convolutional layer
  model.add(tf.keras.layers.Conv2D(
      filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
  model.add(tf.keras.layers.Dropout(0.25))
    
  model.add(tf.keras.layers.Flatten())
    
  model.add(tf.keras.layers.Dense(512, activation='relu'))
  model.add(tf.keras.layers.Dropout(0.5))

  model.add(tf.keras.layers.Dense(NUM_CLASSES))
  model.add(tf.keras.layers.Activation('softmax'))
  
  opt = tf.keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
  return model

## 5. Train, Evaluate and Export ML Models

### a. Set HParam and RunConfig

In [ ]:
NUM_EPOCHS = 1
BATCH_SIZE = 200
TRAIN_SIZE = 50000
NUM_EVAL = 1
CHECKPOINT_STEPS = int((TRAIN_SIZE/BATCH_SIZE) * (NUM_EPOCHS/NUM_EVAL))

hparams = tf.contrib.training.HParams(
  num_epochs=NUM_EPOCHS,
  batch_size=BATCH_SIZE,
)

model_dir = 'trained_models/{}'.format(MODEL_NAME)

run_config = tf.contrib.learn.RunConfig(
  save_checkpoints_steps=CHECKPOINT_STEPS,
  tf_random_seed=19851211,
  model_dir=model_dir
)

print(hparams)
print("Model Directory:", run_config.model_dir)
print("")
print("Dataset Size:", TRAIN_SIZE)
print("Batch Size:", BATCH_SIZE)
print("Steps per Epoch:",TRAIN_SIZE/BATCH_SIZE)
print("Total Steps:", (TRAIN_SIZE/BATCH_SIZE)*NUM_EPOCHS)
print("Required Evaluation Steps:", NUM_EVAL) 
print("That is 1 evaluation step after each",NUM_EPOCHS/NUM_EVAL," epochs")
print("Save Checkpoint After",CHECKPOINT_STEPS,"steps")

### b. Define Serving Function

In [ ]:
def serving_input_fn():

  receiver_tensor = {'images': tf.placeholder(shape=[None, 32, 32, 3], dtype=tf.float32)}
  features = {'images': tf.map_fn(preprocess_image, receiver_tensor['images'])}
  
  return tf.estimator.export.ServingInputReceiver(features, receiver_tensor)

### c. Train and Evaluate

In [ ]:
estimator = tf.keras.estimator.model_to_estimator(keras_model=get_model(), model_dir=model_dir)

# TF.Keras seems not to support exporter...
exporter = None

train_spec = tf.estimator.TrainSpec(
  input_fn=generate_input_fn(file_names=train_data_files,
                             mode=tf.contrib.learn.ModeKeys.TRAIN,
                             num_epochs=hparams.num_epochs,
                             batch_size=hparams.batch_size),
  max_steps=1000,
  hooks=None
)

eval_spec = tf.estimator.EvalSpec(
  input_fn=generate_input_fn(file_names=valid_data_files,
                             mode=tf.contrib.learn.ModeKeys.EVAL,
                             num_epochs=hparams.num_epochs,
                             batch_size=hparams.batch_size),
  steps=50,
  name=None,
  hooks=None,
  exporters=exporter, # Iterable of Exporters, or single one or None.
  start_delay_secs=120,
  throttle_secs=600
)

In [ ]:
if not USE_CHECKPOINT:
  print("Removing previous artifacts...")
  shutil.rmtree(model_dir, ignore_errors=True)

tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

## 6. Evaluate the Model

In [ ]:
train_size = 1000
valid_size = 1000
test_size = 1000

train_input_fn = generate_input_fn(file_names=train_data_files,
                                   mode=tf.contrib.learn.ModeKeys.TRAIN,
                                   num_epochs=None,
                                   batch_size=train_size)

valid_input_fn = generate_input_fn(file_names=valid_data_files,
                                   mode=tf.contrib.learn.ModeKeys.EVAL,
                                   num_epochs=None,
                                   batch_size=valid_size)

test_input_fn = generate_input_fn(file_names=test_data_files,
                                  mode=tf.contrib.learn.ModeKeys.EVAL,
                                  num_epochs=None,
                                  batch_size=test_size)

train_results = estimator.evaluate(input_fn=train_input_fn, steps=1)
print()
print("######################################################################################")
print("# {}".format(train_results))
print("######################################################################################")

valid_results = estimator.evaluate(input_fn=valid_input_fn, steps=1)
print()
print("######################################################################################")
print("# {}".format(valid_results))
print("######################################################################################")

test_results = estimator.evaluate(input_fn=test_input_fn, steps=1)
print()
print("######################################################################################")
print("# {}".format(test_results))
print("######################################################################################")

## 7. Prediction

In [ ]:
test_results = estimator.predict(input_fn=test_input_fn)

for i in test_results:
  print(i)
print()
print("######################################################################################")
print("# {}".format(test_results))
print("######################################################################################")

In [ ]:
import os

export_dir = model_dir + '/export/Servo/'
saved_model_dir = os.path.join(export_dir, os.listdir(export_dir)[-1]) 

print(saved_model_dir)
print('')

predictor_fn = tf.contrib.predictor.from_saved_model(
  export_dir = saved_model_dir,
  signature_def_key='predictions')

N = 1000
labels = []
images = []

for i in range(N):
  result = extract_data(i)
  images.append(result['image'])
  labels.append(result['label'][0])

output = predictor_fn(
  {
    'images': images,
  }
)

In [ ]:
np.sum([a==r for a, r in zip(labels, output['classes'])]) / float(N)

## 8. TensorBoard

In [ ]:
from google.datalab.ml import TensorBoard
TensorBoard().start(model_dir)

In [ ]:
for pid in TensorBoard.list()['pid']:
    TensorBoard().stop(pid)
    print 'Stopped TensorBoard with pid {}'.format(pid)

## (Optional) Using tf.keras for the Inference Part

In [ ]:
def inference(images, mode):
  if mode == tf.estimator.ModeKeys.TRAIN:
    tf.keras.backend.set_learning_phase(True)
  else:
    tf.keras.backend.set_learning_phase(False)
        
  model = tf.keras.models.Sequential()
  # Define input tensor in Keras world.
  model.add(tf.keras.layers.InputLayer(input_tensor=images))
    
  # The first convolutional layer.
  model.add(tf.keras.layers.Conv2D(
      filters=32, kernel_size=(3, 3), padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'))

  # The second convolutional layer.
  model.add(tf.keras.layers.Conv2D(
      filters=32, kernel_size=(3, 3), padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'))
  model.add(tf.keras.layers.Dropout(0.25))
    
  # The third convolutional layer
  model.add(tf.keras.layers.Conv2D(
      filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
    
  # The fourth convolutional layer
  model.add(tf.keras.layers.Conv2D(
      filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
  model.add(tf.keras.layers.Dropout(0.25))
    
  model.add(tf.keras.layers.Flatten())
    
  model.add(tf.keras.layers.Dense(512, activation='relu'))
  model.add(tf.keras.layers.Dropout(0.5))

  model.add(tf.keras.layers.Dense(NUM_CLASSES))
    
  logits = model.output
    
  return logits

## (Optional) Using gRPC to get prediction

In [ ]:
from __future__ import print_function

from grpc.beta import implementations
import numpy
import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2

def predict_with_grpc():
  """                                                                                                                                     
  Note that you are running TensorFlow Serving with below commands.                                                                       
  tensorflow_model_server --port=9000 --model_name=cnn-model-01 --model_base_path=./cnn-model-01                                          

  In addition, make sure cnn-model-01 directory is organized as follows:                                                                  

  cnn-model-01/:                                                                                                                          
  {random_value}                                                                                                                              

  cnn-model-01/{randam_value}/:                                                                                                           
  saved_model.pb  variables                                                                                                               
  """
  host = 'localhost'
  port = '9000'
  channel = implementations.insecure_channel(host, int(port))
  stub = prediction_service_pb2.beta_create_PredictionService_stub(channel)

  result = extract_data(0)
  request = predict_pb2.PredictRequest()
  request.model_spec.name = 'cnn-model-01'
  request.model_spec.signature_name = 'predictions'
  image = result['image']
  label = result['label']
  request.inputs['images'].CopyFrom(
      tf.contrib.util.make_tensor_proto(image, shape=[1, 32, 32, 3]))

  result_future = stub.Predict.future(request, 5.0)
  print(result_future.result().outputs['classes'].int64_val)
  print(result_future.result().outputs['probabilities'].float_val)